# Overall Energy Demand Notebook: Exploring how a homes overall energy demand is attributable to at-home EV charging

<p>We will be using Pecan Street Inc. data from dataport to calculate how much overall energy demand is used in homes by electric vehicle charging.<br><br>
Pecans Streets data can be obtained by applying for a dataport account at https://www.dataport.pecanstreet.org.</p>

In [ ]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
from config.read_config import get_database_config
import numpy as np
import sys
%matplotlib inline
sys.executable  # shows you your path to the python you're using

In [ ]:
# read in db credentials from config/config.txt
# * make sure you add those to the config/config.txt file! *

database_config = get_database_config("./config/config.txt")

In [ ]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format(database_config['username'],
                                                                     database_config['password'],
                                                                     database_config['hostname'],
                                                                     database_config['port'],
                                                                     database_config['database']
                                                                     ))

In [ ]:
#Select a list of Texas homes from dataport metadata having CAR and also has data for year 2018.
query = """select distinct dataid from other_datasets.metadata 
                                          where car1='yes' and grid='yes'
                                          and egauge_1min_min_time < '2018-01-01' 
                                          and egauge_1min_max_time > '2019-01-01'
                                          and state='Texas'
                                          and (egauge_1min_data_availability like '100%' 
                                               or 
                                               egauge_1min_data_availability like '99%')
                                               LIMIT 25;
         """

df = pd.read_sql_query(sqla.text(query), engine)

In [ ]:
# grab dataids and convert them to a string to put into the SQL query
dataids_list = df['dataid'].tolist()
print("{} dataids selected listed here:".format(len(dataids_list)))
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str
dataids_list

In [ ]:
#Check data completeness for dataids selected from metadata above.

query2 = """select dataid,count(*) total_rec from electricity.eg_realpower_1min 
            where dataid in ({})""".format(dataids_str)
query2 = query2 + """ and localminute >= '2018-01-01' and localminute < '2019-01-01' group by 1"""

df2 = pd.read_sql_query(sqla.text(query2), engine)

In [ ]:
#Select homes with atleast 99% data availability for year 2018.
df2['perc'] = (df2['total_rec']/525600)*100
final_dataids = df2[df2['perc'] >= 99]
final_dataids['dataid'].count()

In [ ]:
# Pull data for homes
final_dataids_list = final_dataids['dataid'].tolist()
print("{} dataids selected listed here:".format(len(final_dataids_list)))
final_dataids_str = ','.join(list(map(str, final_dataids_list)))
final_dataids_str
final_dataids_list

In [ ]:

data_pull = """select localminute::timestamp,car1,grid,solar 
               from electricity.eg_realpower_1min 
               where localminute >= '2018-03-01' and localminute <  '2018-06-01' """
data_pull = data_pull + """AND dataid in ({})""".format(final_dataids_str)

data_df = pd.read_sql_query(sqla.text(data_pull), engine)

In [ ]:
#replace null with 0's
day_df = data_df.fillna(0)
# convert localminute to pandas daytime type
day_df['datetime'] = pd.to_datetime(day_df['localminute'])

day_df = day_df.set_index('datetime')

#create day column
day_df['day'] = day_df.index.day

#create new dataframes with grid, solar, car1, and day column
day_df_new = pd.DataFrame(day_df, columns = ['grid', 'solar', 'car1', 'day'])

# group data based on day and get avg
day_df_grouped = day_df_new.groupby(['day']).mean()

day_df_grouped['demand'] = day_df_grouped['car1'] / (day_df_grouped['grid']+ day_df_grouped['solar']) * 100

day_df_grouped.head(10)


In [ ]:
print(day_df_grouped['demand'].mean())

In [ ]:
#replace null with 0's
week_df = data_df.fillna(0)
# convert localminute to pandas daytime type
week_df['datetime'] = pd.to_datetime(week_df['localminute'])

week_df = week_df.set_index('datetime')

#create day column
week_df['week'] = week_df.index.week

#create new dataframes with grid, car1, and week column
week_df_new = pd.DataFrame(week_df, columns = ['grid', 'solar', 'car1', 'week'])

# group data based on week and get avg
week_df_grouped = week_df_new.groupby(['week']).mean()

week_df_grouped['demand'] = week_df_grouped['car1'] / (week_df_grouped['grid']+week_df_grouped['solar']) * 100

week_df_grouped.head(10)

In [ ]:
print(week_df_grouped['demand'].mean())

# Observations:
As seen above daily and weekly overall demand of EV chargers stays pretty consistant. On average both daily and week demand is a little over 12%. At most demand peaks at about 16% and at min deman peaks at 10%.